In [12]:
import json
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [13]:
!pip install matplotlib

In [21]:
# Pipeline with Scaledown
stages_id = ["scaledown", "crop", "mirrorx", "mirrory", "bw", "rotate", "watermark"]
stages_suffix = ["_resized", "_cropped", "_mirrorx", "_mirrory", "_bw", "_rot", "_watermarked"]

# Pipeline without Scaledown
stages_id = ["crop", "mirrorx", "mirrory", "bw", "rotate", "watermark"]
stages_suffix = ["_cropped", "_mirrorx", "_mirrory", "_bw", "_rot", "_watermarked"]

# Version 3 with scaledown in between
version="step-v2"
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "mirror_2rotate", "mirror_3rotate"]
stages_suffix = ["_cropped", "_resized", "_mirror", "_mirror_bw", "_mirror_rot_rot", "_mirror_rot_rot_rot_watermarked"]
moviepy_op_count = [1, 1, 1, 2, 3, 5]


# Version step v3

# stages_helper = ["crop", "scaledown(input for next)", "mirror", "mirror_bw", "mirror_bw_rotate", "mirror_bw_rotate_watermark"]
version="step-v3"
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "mirror_bw_rotate", "mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "", "", "", ""]
powerValues = [2048,4096,8192]

# less-memory version
version="step-v3-512mb-memory"
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "mirror_bw_rotate", "mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "", "", "", ""]
powerValues = [512]

# multiple-memory version crop->scaledown->mirror_bw->rotate->watermark
version="step-v3"
stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror", "_bw", "_rot", "_watermarked"]
powerValues = [512, 1024, 2048, 4096]

# multiple-memory version: crop->scaledown->(mirror,bw,rotate,watermark)
# version="step-v3"
# stages_id = ["crop", "scaledown", "mirror_bw_rotate", "mirror_bw_rotate_watermark"]
# stages_suffix = ["_cropped", "_resized", "", ""]
# powerValues = [1024, 2048, 4096, 8192]

# version="step-noS3"
# stages_id = ["app"]
# stages_suffix = [""]
# powerValues = [2048, 4096, 8192]

# version="step-v3"
# stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "rotate", "watermark"]
# stages_suffix = ["_cropped", "_resized", "_mirror", "_bw", "_rot", "_watermarked"]
# powerValues = [512]


In [22]:
filename = "ElephantsDream"
for stage, suffix in zip(stages_id, stages_suffix):
    input = {}

    input["lambdaARN"] = "arn:aws:lambda:us-west-2:075165449331:function:pipeline-{}-{}".format(stage, version)
    input["powerValues"] = powerValues
    input["num"] = 30
    input["parallelInvocation"] = "true"
    input["strategy"] = "cost"

    input["payload"] = {}
    input["payload"]["filename"] = filename

    filename = filename + suffix

    print(json.dumps(input))

    with open('./../conf/multi-mem/{}.json'.format(stage), 'w') as f:
        json.dump(input, f)

    


{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-crop-step-v3", "powerValues": [512, 1024, 2048, 4096], "num": 30, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-scaledown-step-v3", "powerValues": [512, 1024, 2048, 4096], "num": 30, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-mirror-step-v3", "powerValues": [512, 1024, 2048, 4096], "num": 30, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped_resized"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-bw-step-v3", "powerValues": [512, 1024, 2048, 4096], "num": 30, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped_resized_mirror"}}
{"lambdaARN": "arn:aws:lambd

In [43]:
total_lambda_cost = 0
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "rotate", "watermark"]
powerValues = [1024,2048,4096,8192]
stats = {1024:0, 2048:0, 4096:0, 8192:0}
for stage in stages_id: 
    f = open('./../output/step-v4-multi-memory_2/{}.json'.format(stage), 'r')
    data = json.load(f)
    for stat in data['stats']:
        stats[stat['value']] += stat['averagePrice']
    f.close()

In [44]:
stats

{1024: 0.014253472800000002,
 2048: 0.013942555200000002,
 4096: 0.0155385216,
 8192: 0.0217444416}

In [ ]:
total_lambda_cost = 0
stages_id = ["app"]
powerValues = [1024,2048,4096]
stats = {1024:0, 2048:0, 4096:0}
for stage in stages_id: 
    f = open('./../output/step-v4-512mb-mem/{}.json'.format(stage), 'r')
    data = json.load(f)
    for stat in data['stats']:
        stats[stat['value']] += stat['averagePrice']
    f.close()

In [41]:
total_lambda_cost = 0
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "rotate", "watermark"]
powerValues = [1024,2048,4096,8192]
stats = {512:0}
for stage in stages_id: 
    f = open('./../output/step-v4-512mb-mem/{}.json'.format(stage), 'r')
    data = json.load(f)
    for stat in data['stats']:
        stats[stat['value']] += stat['averagePrice']
    f.close()

In [42]:
stats

{512: 0.014506195200000002}

In [ ]:
total_lambda_cost = 0
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "rotate", "watermark"]
powerValues = [1024,2048,4096,8192]
stats = {512:0}
for stage in stages_id: 
    f = open('./../output/step-v4-multi-memory_1_aggregatedSteps/{}.json'.format(stage), 'r')
    data = json.load(f)
    for stat in data['stats']:
        stats[stat['value']] += stat['averagePrice']
    f.close()

In [47]:
total_lambda_cost = 0
stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
powerValues = [1024,2048,4096,8192]
stats = {1024:0, 2048:0, 4096:0, 8192:0}
for stage in stages_id:
    f = open('./../output/step-v4-multi-memory_1_aggregatedSteps/{}.json'.format(stage), 'r')
    data = json.load(f)
    for stat in data['stats']:
        stats[stat['value']] += stat['averagePrice']
    f.close()

In [48]:
stats

{1024: 0.012926172000000003,
 2048: 0.012779793600000002,
 4096: 0.014063280000000001,
 8192: 0.0200088}